# Mini-projekt NLP w PyTorch
Ten notatnik pokazuje najważniejsze koncepcje związane z przetwarzaniem języka naturalnego (NLP) w PyTorch na bardzo małym, sztucznym zbiorze danych. Przechodzimy przez cały proces: od przygotowania tekstu i budowy słownika, przez kodowanie sekwencji, aż po trenowanie prostej sieci RNN i wykonywanie prognoz.

Postaramy dokonać analizy sentymentu - czyli określić, czy dany tekst jest pozytywny czy negatywny.

## 1. Importy i ustawienia
Zaczynamy od zaimportowania PyTorcha i kilku pomocniczych bibliotek. Ustawiamy także ziarno losowe, aby wyniki były powtarzalne.

In [40]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

import numpy as np
import random

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)


## 2. Mini zbiór danych
Dla przejrzystości wykorzystamy niewielki, ręcznie przygotowany zbiór zdań oznaczonych etykietami `1` (pozytywne) lub `0` (negatywne). Dzięki temu możemy szybko podejrzeć cały zbiór i łatwiej zrozumieć każdy krok przygotowania danych.

In [41]:
samples = [
    ("kocham ten film", 1),
    ("uwielbiam tę książkę", 1),
    ("ten kurs jest świetny", 1),
    ("co za wspaniały dzień", 1),
    ("jestem zachwycony", 1),
    ("nienawidzę tej gry", 0),
    ("to był stracony czas", 0),
    ("film był okropny", 0),
    ("jestem rozczarowany", 0),
    ("fatalna obsługa", 0)
]

for text, label in samples:
    print(f"{label} -> {text}")


1 -> kocham ten film
1 -> uwielbiam tę książkę
1 -> ten kurs jest świetny
1 -> co za wspaniały dzień
1 -> jestem zachwycony
0 -> nienawidzę tej gry
0 -> to był stracony czas
0 -> film był okropny
0 -> jestem rozczarowany
0 -> fatalna obsługa


## 3. Budowa słownika (vocabulary)
Modelom NLP trudno pracować bezpośrednio na surowych słowach. Potrzebujemy mapowania słów na liczby całkowite. Budujemy słownik zawierający:

- `PAD` – specjalny token wypełniający (używany przy wyrównywaniu długości sekwencji),
- `UNK` – token dla słów nieznanych (gdyby pojawiły się inne słowa w nowych zdaniach),
- wszystkie słowa z naszego mini zbioru.

Tokenizacja będzie bardzo prosta – rozdzielamy słowa po spacjach i zamieniamy na małe litery.

In [42]:
from collections import Counter

def tokenize(text):
    return text.lower().split()

counter = Counter()
for text, _ in samples:
    counter.update(tokenize(text))

vocab = {"<PAD>": 0, "<UNK>": 1}
for word in counter:
    vocab[word] = len(vocab)

inverse_vocab = {idx: word for word, idx in vocab.items()}

print("Rozmiar słownika:", len(vocab))
print(vocab)


Rozmiar słownika: 28
{'<PAD>': 0, '<UNK>': 1, 'kocham': 2, 'ten': 3, 'film': 4, 'uwielbiam': 5, 'tę': 6, 'książkę': 7, 'kurs': 8, 'jest': 9, 'świetny': 10, 'co': 11, 'za': 12, 'wspaniały': 13, 'dzień': 14, 'jestem': 15, 'zachwycony': 16, 'nienawidzę': 17, 'tej': 18, 'gry': 19, 'to': 20, 'był': 21, 'stracony': 22, 'czas': 23, 'okropny': 24, 'rozczarowany': 25, 'fatalna': 26, 'obsługa': 27}


## 4. Kodowanie i wyrównywanie sekwencji
Każde zdanie zamieniamy na listę indeksów słów. Następnie wyrównujemy długość sekwencji do stałej wartości `MAX_LEN`, dopełniając krótsze zdania tokenem `<PAD>`. Dzięki temu możemy umieścić dane w jednej macierzy tensora.

W razie napotkania nieznanego słowa skorzystamy z `UNK`, choć w tym przykładzie wszystkie słowa są znane.

In [43]:
MAX_LEN = 5

def encode(text, vocab, max_len):
    tokens = tokenize(text)
    ids = [vocab.get(tok, vocab["<UNK>"]) for tok in tokens]
    if len(ids) < max_len:
        ids += [vocab["<PAD>"]] * (max_len - len(ids))
    else:
        ids = ids[:max_len]
    return ids

encoded_texts = [encode(text, vocab, MAX_LEN) for text, _ in samples]
labels = [label for _, label in samples]

print("Przykładowa sekwencja:", encoded_texts[0], "=", [inverse_vocab[idx] for idx in encoded_texts[0]])


Przykładowa sekwencja: [2, 3, 4, 0, 0] = ['kocham', 'ten', 'film', '<PAD>', '<PAD>']


## 5. Dataset i DataLoader
Korzystamy ze standardowej struktury PyTorch: tworzymy klasę `Dataset`, która zwraca pary `(tensor_wejściowy, etykieta)`. Następnie pakujemy dane w `DataLoader`, aby łatwo iterować po mini-batchach podczas treningu.

In [44]:
class SentimentDataset(Dataset):
    def __init__(self, encoded_texts, labels):
        self.encoded = torch.tensor(encoded_texts, dtype=torch.long)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.encoded[idx], self.labels[idx]

full_dataset = SentimentDataset(encoded_texts, labels)

# Prosty podział: 8 przykładów treningowych, 2 testowe
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [8, 2], generator=torch.Generator().manual_seed(SEED))

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2)

print('Rozmiar zbioru treningowego:', len(train_dataset))
print('Rozmiar zbioru testowego:', len(test_dataset))


Rozmiar zbioru treningowego: 8
Rozmiar zbioru testowego: 2


## 6. Model RNN
Model składa się z trzech elementów:
1. `nn.Embedding` – zamienia indeksy słów na gęste wektory (embeddingi).
2. `nn.RNN` – przetwarza sekwencję i zwraca stany ukryte.
3. `nn.Linear` + `nn.Sigmoid` – mapuje ostatni stan ukryty na prawdopodobieństwo klasy `1`.

Utrzymujemy model możliwie prosty, aby skupić się na przepływie danych i interpretacji wyników.

In [45]:
class SimpleSentimentRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim=16, hidden_size=32):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim, padding_idx=0)
        self.rnn = nn.RNN(input_size=embed_dim, hidden_size=hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded)
        # hidden ma kształt (num_layers, batch, hidden_size)
        last_hidden = hidden[-1]
        logits = self.fc(last_hidden)
        probs = self.sigmoid(logits)
        return probs.squeeze(1)

model = SimpleSentimentRNN(vocab_size=len(vocab))
model


SimpleSentimentRNN(
  (embedding): Embedding(28, 16, padding_idx=0)
  (rnn): RNN(16, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

## 7. Trening
Używamy binarnej funkcji straty (`BCELoss`) oraz optymalizatora Adam. Ponieważ danych jest mało, wystarczy kilkanaście epok, aby model nauczył się prostych reguł. W każdej epoce wypisujemy stratę i dokładność na zbiorze treningowym.

In [46]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

EPOCHS = 20
for epoch in range(1, EPOCHS + 1):
    model.train()
    epoch_loss = 0.0
    correct = 0
    total = 0

    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * inputs.size(0)
        preds = (outputs >= 0.5).float()
        correct += (preds == targets).sum().item()
        total += targets.size(0)

    avg_loss = epoch_loss / total
    acc = correct / total
    print(f"Epoka {epoch:02d} | Strata: {avg_loss:.4f} | Dokładność: {acc:.2f}")


Epoka 01 | Strata: 0.7180 | Dokładność: 0.38
Epoka 02 | Strata: 0.5905 | Dokładność: 0.75
Epoka 03 | Strata: 0.4511 | Dokładność: 1.00
Epoka 04 | Strata: 0.2809 | Dokładność: 1.00
Epoka 05 | Strata: 0.1425 | Dokładność: 1.00
Epoka 06 | Strata: 0.0680 | Dokładność: 1.00
Epoka 07 | Strata: 0.0377 | Dokładność: 1.00
Epoka 08 | Strata: 0.0213 | Dokładność: 1.00
Epoka 09 | Strata: 0.0129 | Dokładność: 1.00
Epoka 10 | Strata: 0.0086 | Dokładność: 1.00
Epoka 11 | Strata: 0.0060 | Dokładność: 1.00
Epoka 12 | Strata: 0.0045 | Dokładność: 1.00
Epoka 13 | Strata: 0.0034 | Dokładność: 1.00
Epoka 14 | Strata: 0.0027 | Dokładność: 1.00
Epoka 15 | Strata: 0.0022 | Dokładność: 1.00
Epoka 16 | Strata: 0.0019 | Dokładność: 1.00
Epoka 17 | Strata: 0.0016 | Dokładność: 1.00
Epoka 18 | Strata: 0.0015 | Dokładność: 1.00
Epoka 19 | Strata: 0.0013 | Dokładność: 1.00
Epoka 20 | Strata: 0.0012 | Dokładność: 1.00


## 8. Ewaluacja na zbiorze testowym
Sprawdzamy, jak model radzi sobie na dwóch przykładach testowych, które nie brały udziału w treningu.

In [47]:
model.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        preds = (outputs >= 0.5).float()
        test_correct += (preds == targets).sum().item()
        test_total += targets.size(0)

print(f'Dokładność na teście: {test_correct / test_total:.2f} ({test_correct}/{test_total})')


Dokładność na teście: 0.50 (1/2)


## 9. Predykcje dla nowych zdań
Przygotowujemy pomocniczą funkcję, która przyjmuje tekst, koduje go tak jak wcześniej, a następnie zwraca prawdopodobieństwo klasy pozytywnej. Dzięki temu możemy szybko sprawdzić, jak model reaguje na nowe wypowiedzi.

In [48]:
def predict_sentiment(model, text):
    model.eval()
    encoded = torch.tensor([encode(text, vocab, MAX_LEN)], dtype=torch.long)
    with torch.no_grad():
        prob = model(encoded).item()
    return prob

examples = [
    "kocham tę grę",
    "co za fatalny dzień",
    "jestem bardzo zadowolony",
    "strasznie nudny film"
]

for sentence in examples:
    prob = predict_sentiment(model, sentence)
    print(f"{sentence!r} -> prawdopodobieństwo klasy pozytywnej: {prob:.2f}")


'kocham tę grę' -> prawdopodobieństwo klasy pozytywnej: 1.00
'co za fatalny dzień' -> prawdopodobieństwo klasy pozytywnej: 0.04
'jestem bardzo zadowolony' -> prawdopodobieństwo klasy pozytywnej: 0.99
'strasznie nudny film' -> prawdopodobieństwo klasy pozytywnej: 0.58


## 10. Podsumowanie
W tym mini-projekcie omówiliśmy podstawowy przepływ pracy w NLP z użyciem PyTorcha:

1. **Przygotowanie danych** – tokenizacja, budowa słownika, kodowanie i wyrównanie sekwencji.
2. **Struktura danych w PyTorch** – `Dataset` i `DataLoader` ułatwiają iterację po mini-batchach.
3. **Model** – `Embedding` + `RNN` + warstwa gęsta generująca prawdopodobieństwo klasy.
4. **Trening i ewaluacja** – klasyczna pętla treningowa z funkcją straty `BCELoss` oraz pomiarem dokładności.
5. **Predykcje** – prosty interfejs do oceniania nowych zdań.

Chociaż przykład jest niewielki, przedstawia wszystkie najważniejsze elementy pipeline'u NLP. Przy większych projektach wystarczy wymienić zbiór danych, rozszerzyć preprocessing (np. usuwanie znaków, stemming), zastosować bogatszą architekturę (GRU/LSTM/Transformer) i zwiększyć rozmiar embeddingów.

## 11. Rozszerzenie: proste techniki poprawy modelu
Na koniec prezentujemy dwie techniki, które często poprawiają wyniki modeli NLP, nawet w bardzo prostych scenariuszach:

- **Augmentacja danych** – tworzymy dodatkowe przykłady na podstawie istniejących, np. zamieniając niektóre słowa na synonimy.
- **Walidacja krzyżowa** – uczymy model na kilku podziałach danych i uśredniamy wyniki, dzięki czemu ocena jest bardziej stabilna na małych zbiorach.

### 11.1 Augmentacja danych (synonimy/perturbacje)
W NLP augmentacja danych polega na sztucznym rozszerzaniu zbioru treningowego. W tym przykładzie losowo wybierzemy zdanie pozytywne i zamienimy w nim jedno słowo na prosty synonim (utrzymujemy minimalizm kodu). Celem jest pokazanie koncepcji – na poważne projekty warto używać bardziej rozbudowanych narzędzi (np. WordNet, bibliotek typu `nlpaug`).

In [49]:
import random

synonym_dict = {
    "kocham": ["uwielbiam", "bardzo lubię"],
    "wspaniały": ["świetny", "znakomity"],
    "fatalna": ["okropna", "beznadziejna"],
    "nienawidzę": ["nie cierpię"],
}

augmented_samples = []
for text, label in samples:
    tokens = tokenize(text)
    new_tokens = tokens.copy()
    candidate_positions = [i for i, tok in enumerate(tokens) if tok in synonym_dict]
    if candidate_positions:
        pos = random.choice(candidate_positions)
        new_tokens[pos] = random.choice(synonym_dict[new_tokens[pos]])
        new_text = " ".join(new_tokens)
        augmented_samples.append((new_text, label))

print("Oryginalne i zaugmentowane przykłady:")
for original, augmented in zip(samples, augmented_samples):
    print(f"{original[0]!r} -> {augmented[0]!r}")


Oryginalne i zaugmentowane przykłady:
'kocham ten film' -> 'uwielbiam ten film'
'uwielbiam tę książkę' -> 'co za świetny dzień'
'ten kurs jest świetny' -> 'nie cierpię tej gry'
'co za wspaniały dzień' -> 'okropna obsługa'


Teraz połączymy pierwotny zbiór z zaugmentowanym i ponownie zakodujemy sekwencje. W praktyce należałoby przebudować słownik, aby uwzględniał nowe słowa. Tutaj dla prostoty wykorzystamy istniejący słownik i potraktujemy ewentualne nowe słowa jako `<UNK>`.

In [50]:
augmented_dataset = samples + augmented_samples

encoded_augmented = [encode(text, vocab, MAX_LEN) for text, _ in augmented_dataset]
labels_augmented = [label for _, label in augmented_dataset]

print('Liczba przykładów po augmentacji:', len(encoded_augmented))


Liczba przykładów po augmentacji: 14


### 11.2 Walidacja krzyżowa na małym zbiorze
Na małych zbiorach zwykły podział na trening/test może dawać niestabilne wyniki. **Walidacja krzyżowa (k-fold cross-validation)** dzieli zbiór na `K` fragmentów, z których każdy pełni rolę walidacyjną raz, a pozostałe służą do treningu. Uśredniamy wyniki, aby zredukować wpływ losowego podziału.

In [51]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=3, shuffle=True, random_state=SEED)

encoded_tensor = torch.tensor(encoded_augmented, dtype=torch.long)
labels_tensor = torch.tensor(labels_augmented, dtype=torch.float32)

fold_results = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(encoded_tensor)):
    train_subset = SentimentDataset(encoded_tensor[train_idx], labels_tensor[train_idx])
    val_subset = SentimentDataset(encoded_tensor[val_idx], labels_tensor[val_idx])

    train_loader_cv = DataLoader(train_subset, batch_size=4, shuffle=True)
    val_loader_cv = DataLoader(val_subset, batch_size=4)

    cv_model = SimpleSentimentRNN(len(vocab))
    criterion = nn.BCELoss()
    optimizer = optim.Adam(cv_model.parameters(), lr=0.01)

    for epoch in range(5):
        cv_model.train()
        for inputs, targets in train_loader_cv:
            optimizer.zero_grad()
            outputs = cv_model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

    cv_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in val_loader_cv:
            outputs = cv_model(inputs)
            preds = (outputs >= 0.5).float()
            correct += (preds == targets).sum().item()
            total += targets.size(0)

    acc = correct / total
    fold_results.append(acc)
    print(f'Fold {fold + 1} | Dokładność walidacyjna: {acc:.2f}')

print('Średnia dokładność z walidacji krzyżowej:', np.mean(fold_results))


Fold 1 | Dokładność walidacyjna: 0.80
Fold 2 | Dokładność walidacyjna: 0.40
Fold 3 | Dokładność walidacyjna: 1.00
Średnia dokładność z walidacji krzyżowej: 0.7333333333333334


/var/folders/ff/6wst9f1n1g96lc__ld426szh0000gn/T/ipykernel_39040/567183548.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.encoded = torch.tensor(encoded_texts, dtype=torch.long)
/var/folders/ff/6wst9f1n1g96lc__ld426szh0000gn/T/ipykernel_39040/567183548.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.float32)


## 12. Co nam dają te techniki?
- **Augmentacja** zwiększa różnorodność danych, dzięki czemu model może nauczyć się bardziej ogólnych wzorców (np. rozpoznawać synonimy). W realnych projektach często korzysta się z bardziej zaawansowanych metod – zamiany słów na synonimy, losowe usuwanie/ dodawanie słów, zamiany zdań na równoważne itp.
- **Walidacja krzyżowa** dostarcza stabilniejszej oceny jakości modelu – zwłaszcza na małej liczbie przykładów. Uśredniony wynik lepiej odzwierciedla realną skuteczność niż pojedynczy podział na trening/test.

Obie techniki są tanim sposobem na zwiększenie odporności modeli NLP na małych zbiorach i warto je mieć w zestawie narzędzi.